In [1]:
import pandas as pd
from functools import partial
from ctcv_helper import print_table
from ctcv_helper import read_dataframe
from ctcv_helper import read_dataframe_ratio

In [2]:
# Read limits from csv files
limits = {}
for model in ['K4', 'K5', 'K6']:
    df = pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_0p5.csv"%model, sep=",", index_col=None)
    df = df.append(pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_1p0.csv"%model, sep=",", index_col=None), ignore_index=True)
    df = df.append(pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_1p5.csv"%model, sep=",", index_col=None), ignore_index=True)
    limits[model] = df

In [3]:
# Read the cross sections from csv files
df_xsbr = {}
df_br = {}
df_xsecs = {}
for model in ['K4', 'K5', 'K6']:
    df_xsbr[model]  = pd.DataFrame.from_csv("xsbr_scalings_%s.csv"%model, sep=",", index_col=None)
    df_br[model]    = pd.DataFrame.from_csv("br_scalings_%s.csv"%model,   sep=",", index_col=None)
    df_xsecs[model] = pd.DataFrame.from_csv("xsecs_tH_ttH_%s.csv"%model, sep=",", index_col=None)

In [12]:
# Print pure limits on r
MOD = 'K6'
print " alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5"
print_table([partial(read_dataframe, df=limits[MOD], att='exp')],
             linepat=" %5.4f ")

 alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5
-0.973 -6.000:  0.0356                 
-0.941 -4.000:  0.0356                 
-0.900 -3.000:  0.0708  0.0708         
-0.862 -2.500:  0.1099                 
-0.800 -2.000:  0.1821  0.1821  0.1821 
-0.692 -1.500:  0.3325  0.3325         
-0.640 -1.333:                  0.4185 
-0.610 -1.250:          0.4678         
-0.500 -1.000:  0.6763  0.6763  0.6763 
-0.410 -0.833:                  0.8809 
-0.360 -0.750:          1.0015         
-0.308 -0.667:                  1.1416 
-0.200 -0.500:  1.5073  1.5068  1.5068 
-0.100 -0.333:                  1.9854 
-0.059 -0.250:          2.2520         
-0.027 -0.167:                  2.5498 
 0.000  0.000:  3.3213  3.3213  3.3203 
 0.027  0.167:                  4.3516 
 0.059  0.250:          4.8672         
 0.100  0.333:                  5.1895 
 0.200  0.500:  4.4219  4.4219  4.4219 
 0.308  0.667:                  2.8848 
 0.360  0.750:          2.3262         
 0.410  0.833:                  1.8906 
 

In [14]:
# Print limit * cross section * BR
def xs_limit(ct,cv):
    # Take the cross section for cv=1.0, ct=ct/cv
    xsec = read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs[MOD], att='tot')
    lim  = read_dataframe(ct, cv, df=limits[MOD], att='exp')
    return float(xsec)*float(lim)

print " alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5"
print_table([xs_limit], linepat=" %5.4f ")

 alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5
-0.973 -6.000:  0.3168                 
-0.941 -4.000:  0.3168                 
-0.900 -3.000:  0.3136  0.3136         
-0.862 -2.500:  0.3141                 
-0.800 -2.000:  0.3104  0.3104  0.3104 
-0.692 -1.500:  0.3068  0.3068         
-0.640 -1.333:                  0.3068 
-0.610 -1.250:          0.3043         
-0.500 -1.000:  0.3002  0.3002  0.3002 
-0.410 -0.833:                  0.2971 
-0.360 -0.750:          0.2929         
-0.308 -0.667:                  0.2884 
-0.200 -0.500:  0.2803  0.2802  0.2802 
-0.100 -0.333:                  0.2686 
-0.059 -0.250:          0.2599         
-0.027 -0.167:                  0.2517 
 0.000  0.000:  0.2467  0.2467  0.2466 
 0.027  0.167:                  0.2665 
 0.059  0.250:          0.2868         
 0.100  0.333:                  0.3081 
 0.200  0.500:  0.3050  0.3050  0.3050 
 0.308  0.667:                  0.2631 
 0.360  0.750:          0.2498         
 0.410  0.833:                  0.2407 
 

In [16]:
print_table([lambda ct,cv: read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs[MOD], att='tot')], linepat=" %5.4f ")

-0.973 -6.000:  8.8999                 
-0.941 -4.000:  8.8999                 
-0.900 -3.000:  4.4292  4.4292         
-0.862 -2.500:  2.8584                 
-0.800 -2.000:  1.7047  1.7047  1.7047 
-0.692 -1.500:  0.9227  0.9227         
-0.640 -1.333:                  0.7330 
-0.610 -1.250:          0.6504         
-0.500 -1.000:  0.4439  0.4439  0.4439 
-0.410 -0.833:                  0.3373 
-0.360 -0.750:          0.2925         
-0.308 -0.667:                  0.2526 
-0.200 -0.500:  0.1859  0.1859  0.1859 
-0.100 -0.333:                  0.1353 
-0.059 -0.250:          0.1154         
-0.027 -0.167:                  0.0987 
 0.000  0.000:  0.0743  0.0743  0.0743 
 0.027  0.167:                  0.0612 
 0.059  0.250:          0.0589         
 0.100  0.333:                  0.0594 
 0.200  0.500:  0.0690  0.0690  0.0690 
 0.308  0.667:                  0.0912 
 0.360  0.750:          0.1074         
 0.410  0.833:                  0.1273 
 0.500  1.000:  0.1799  0.1799  0.1799 
